## Overview

This notebook shows you how to load data from JDBC databases using Spark SQL.

*For production, you should control the level of parallelism used to read data from the external database, using the parameters described in the documentation.*

### Step 1: Connection Information

This is a **Python** notebook so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` magic command. Python, Scala, SQL, and R are all supported.

First we'll define some variables to let us programmatically create these connections.

In [0]:
pip install json5

Python interpreter will be restarted.
Requirement already satisfied: json5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-41c9f557-09f3-456c-beac-0a7fbc2ad100/lib/python3.7/site-packages (0.9.5)
Python interpreter will be restarted.

In [0]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://queenie.db.elephantsql.com:5432/ycfglres"
user = "ycfglres"
password = "z-PQKACVeHQ_f6wbuQQPpmyDJ55IhNwN"

### Step 2: Reading the data

Now that we specified our file metadata, we can create a DataFrame. You'll notice that we use an *option* to specify that we'd like to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python, notice how we will programmatically reference the variables we defined above.

In [0]:
import json5 as json

#  .option("dbtable", table)\
#  .option("query", "select * from item where item.Id='Bananas'")\
  
print("============== ITEMS FROM POSTGRES ==============")
table = "item"
df_items = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()

print("df_items")
df_items.show()
#df_items.toJSON().collect(): ['{"Id":1,"Name":"Bananas","Desc":"Bananas","Active":"1"}']
items = []
for e in df_items.toJSON().collect():
  e = json.loads(e)
  items.append(e)
print("Items as JSON Array", items)
bc_items = spark.sparkContext.broadcast(items)


print("============== STORE FROM POSTGRES ==============")
table = "store"
df_store = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()

print("df_store")
df_store.show()
stores = []
#['{"Id":1,"Name":"Store-A","City":"New York","State":"New York","Country":"USA","Active":1}']
for e in df_store.toJSON().collect():
  e = json.loads(e)
  stores.append(e)
print("Stores as JSON Array", stores)
bc_stores = spark.sparkContext.broadcast(stores)


print("============== TRANSACTIONS FROM POSTGRES ==============")
table = "transaction"
df_trans = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()
print("df_trans")
df_trans.show()


============== ITEMS FROM POSTGRES ==============

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1472450366695387> in <module> 
 10 . option ( "query" , "select * from item where item.Id='Bananas'" ) \ 
 11 . option ( "user" , user ) \ 
 ---> 12 . option ( "password" , password ) \ 
 13 . load ( ) 
 14 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 182 return self . _df ( self . _jreader . load ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 183 else : 
 --> 184 return self . _df ( self . _jreader . load ( ) ) 
 185 
 186 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o323.load.
: org.postgresql.util.PSQLException: ERROR: column item.id does not exist
 Hint: Perhaps you meant to reference the column "item.Id".
 Position: 41
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2477)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2190)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:300)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:428)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:354)
	at org.postgresql.jdbc.PgPreparedStatement.executeWithFlags(PgPreparedStatement.java:169)
	at org.postgresql.jdbc.PgPreparedStatement.executeQuery(PgPreparedStatement.java:117)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:61)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:226)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:35)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:387)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:387)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:376)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:376)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:261)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

### Step 3: Querying the data

Now that we created our DataFrame. We can query it. For instance, you can select some particular columns to select and display within Databricks.

In [0]:
display(df_items.select("Name"))
display(df_store.select("Name"))
display(df_trans.select("TDesc"))

In [0]:
print("===================== MAP =====================")
import json5 as json
import pyspark.sql.functions as f
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg
from pyspark.sql.types import Row
import json5 as json

def myTransEnrich(row, bc_stores, bc_items):
  print("myTransEnrich::type:%s, data:%s" % (type(row), row))
  #myTransEnrich::type:<class 'pyspark.sql.types.Row'>, data:Row(TId=1, StoreId=3, ItemId=3, TDesc='Oranges for StoreC@DAL', TDate='2021-04-19T02:01:01')
  row_dict = row.asDict()
  print("myTransEnrich::row_dict:", type(row_dict), row_dict)
  #myTransEnrich::row_dict: <class 'dict'> {'TId': 1, 'StoreId': 2, 'ItemId': 1, 'TDesc': 'Bananas for StoreA@NYC', 'TDate': '2021-04-19T03:01:01'}
  itemId = row_dict.get("ItemId")
  storeId = row_dict.get("StoreId")
        
  #print("myTransEnrich::bc_user.value::", type(bc_stores.value), bc_stores.value)
  #myTransEnrich::bc_user.value:: <class 'list'> [{'Id': 1, 'Name': 'Store-A', 'City': 'New York', 'State': 'New York', 'Country': 'USA', 'Active': 1}]
  myitem = list(filter(lambda x:x["Id"]==itemId, bc_items.value))
  print("myTransEnrich::myitem:", (myitem))
  #myTransEnrich::myitem: [{'Id': 3, 'Name': 'Store-C', 'City': 'Dallas', 'State': 'Texas', 'Country': 'USA', 'Active': 1}]
  myitem = myitem[0]
  row_dict["ItemName"] = myitem["Name"]

  #print("myTransEnrich::bc_items.value::", type(bc_items.value), bc_items.value)
  #myTransEnrich::bc_items.value:: <class 'list'> [{'Id': 1, 'Name': 'Bananas', 'Desc': 'Bananas', 'Active': '1'}]
  mystore = list(filter(lambda x:x["Id"]==storeId, bc_stores.value))
  print("myTransEnrich::mystore:", (mystore))
  #myTransEnrich::mystore: [{'Id': 3, 'Name': 'Organes', 'Desc': 'Naval Oranges', 'Active': '1'}]
  mystore = mystore[0]
  row_dict["StoreName"] = mystore["Name"]
  
  return Row(**row_dict)

In [0]:
df_enriched = df_trans.rdd.map(lambda row: myTransEnrich(row, bc_stores, bc_items))
print("df_enriched:0:", df_enriched, df_enriched.toDF())
#df_enriched.toDF(["TId", "StoreId", "ItemId", "TDesc", "TDate"]).show()

table = "transaction_enrich"
df_enriched.toDF().write.format("jdbc").option("driver", driver).option("url", url).option("user", user).option("password", password).option("dbtable", table).saveAsTable("transaction_enrich")

This table will persist across cluster restarts as well as allow various users across different notebooks to query this data. However, this will not connect back to the original database when doing so.